# BASIC FLASK, REST API AND MORE
*Author: Marco Prenassi*   
*License: Creative Commons Attribution 4.0 International*    

In [1]:
!pip install flask

In [3]:
from werkzeug.wrappers import Request, Response
from flask import Flask

app = Flask(__name__)

# Decorator that maps the apps routes, "/" is our host (like: http://localhost.5000/)
@app.route("/")
def hello():
    return "Hello World!"

# Entry point. Remember that you have to interrupt this code
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple("0.0.0.0", 5000, app)

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.88.0.14:5000
Press CTRL+C to quit
192.168.127.1 - - [15/Oct/2024 07:01:28] "GET / HTTP/1.1" 200 -
192.168.127.1 - - [15/Oct/2024 07:01:28] "GET /favicon.ico HTTP/1.1" 404 -


In [1]:
from flask import Flask
from flask import request

app = Flask(__name__)
app.debug = True

# Let's point to a dynamic node with: <variable>
@app.route('/users/<user_id>', methods = ['GET', 'POST', 'DELETE'])
def user(user_id):
    if request.method == 'GET':
        # Let's instantiate a FORM
        raw_html = f"Hi, User {user_id}!<br>"
        # REMEMBER POST must be written post, with lower case letters, also, 
        # let's point our post towards ourselves
        raw_html += f'<form action="/users/{user_id}" method="post">'
        raw_html += '<input type="text" id="laboratory" name="laboratory"><br>'
        raw_html += '<input type="submit" value="Submit">'
        raw_html += '</form>'
        return raw_html
    if request.method == 'POST':
        # Let's read the form, request (object) have all the data of the packet
        data = request.form.get('laboratory') # a multidict containing POST data
        return f"Hi, poster {user_id} - you posted {data}"
    if request.method == 'DELETE':
        return "This is dangerous"
    else:
        return "Error 405 Method Not Allowed"

In [2]:
# Beware that this is really unstable, copy your app into a file and from terminal
# flask --app name_of_the_file run --host="0.0.0.0" [without the .py]
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple("0.0.0.0", 5000, app, use_debugger=True)

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.88.0.14:5000
Press CTRL+C to quit
192.168.127.1 - - [15/Oct/2024 07:06:14] "GET /users/marco HTTP/1.1" 200 -
192.168.127.1 - - [15/Oct/2024 07:06:17] "GET / HTTP/1.1" 404 -


In [108]:
# Let's put all together (FLASK + SQLALCHEMY ORM!)
from flask import Flask
from flask import request
from werkzeug.debug import DebuggedApplication

app = Flask(__name__)
app.debug = True

import sqlalchemy as sql

from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
# We inherit DeclarativeBase to create a Base, why? Because we need to create a specific registry linked to the database.
# class normally do not hold attributes, but in this specific case they do, as it is used as a "legend" to navigate the SQL schema
class Base(DeclarativeBase):
    pass

# Let's see if there's something (you only see an "object" like: <sqlalchemy.orm.decl_api.registry object at ....>
print(Base.registry)

class Sample(Base):
    __tablename__ = "sample"
    name: Mapped[str] = mapped_column(String[50], primary_key=True)
    molecular_weight: Mapped[float]

# Create a new sqlite database
from sqlalchemy import create_engine
engine = create_engine("sqlite:///databases/flask_database.db", echo=False)
# Create all the table, using the definition in Base (that's why we used it)
Base.metadata.create_all(engine)

# Let's populate our table with data!
from sqlalchemy.orm import Session
from sqlalchemy.exc import IntegrityError

# Create a session
def database_insert(name, molecular_weight):
    with Session(engine) as session:
        try:
            # Let's insert the first row of data
            sample = Sample(
                name=name,
                molecular_weight=molecular_weight,
            )
            session.add(sample)
            session.commit()
        except IntegrityError as e:
            print(f"Error, created new data must have a unique primary key! \n{e}")

@app.route('/users/<user_id>', methods = ['GET', 'POST', 'DELETE'])
def user(user_id):
    if request.method == 'GET':
        raw_html = f"Hi, User {user_id}!<br>"
        # REMEMBER POST must be written post, with lower case letters
        raw_html += f'<form action="/users/{user_id}" method="post">'
        raw_html += '<label for="fname">Name:</label><br>'
        raw_html += '<input type="text" id="name" name="name"><br>'
        raw_html += '<label for="fname">Molecular weight:</label><br>'
        raw_html += '<input type="text" id="molecular_weight" name="molecular_weight"><br>'
        raw_html += '<input type="submit" value="Submit">'
        raw_html += '</form>'
        return raw_html
    if request.method == 'POST':
        name = request.form.get('name') # a multidict containing POST data
        molecular_weight = request.form.get('molecular_weight')
        database_insert(name, molecular_weight)
        return f"Hi, poster {user_id} - you posted {name} with molecular weight of {molecular_weight}"
    if request.method == 'DELETE':
        return "This is dangerous"
    else:
        return "Error 405 Method Not Allowed"

In [109]:
# Beware that this is really unstable, copy your app into a file and from terminal
# flask --app name_of_the_file run --host="0.0.0.0" [without the .py]
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple("0.0.0.0", 5000, app, use_debugger=True)

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.88.0.13:5000
Press CTRL+C to quit
192.168.127.1 - - [13/Oct/2024 17:40:31] "GET /users/gianni HTTP/1.1" 200 -
192.168.127.1 - - [13/Oct/2024 17:40:35] "POST /users/gianni HTTP/1.1" 200 -


In [163]:
# Let's use our app as a bridge to other web services, 
# Let's check if they are available
import requests as rq
import json
# rq.get('https://data.rcsb.org/rest/v1/core/assembly/{entry_id}/{assembly_id}')
result = rq.get('https://data.rcsb.org/rest/v1/core/assembly/1RH7/1').json()


{'details': 'author_and_software_defined_assembly', 'id': '1', 'method_details': 'PISA', 'oligomeric_count': 3, 'oligomeric_details': 'trimeric', 'rcsb_candidate_assembly': 'Y', 'rcsb_details': 'author_and_software_defined_assembly'} aa


In [167]:
# Ok, let's bridge our two services
app = Flask(__name__)
app.debug = True

@app.route('/get_entry/', methods = ['GET', 'POST', 'DELETE'])
def protein_retrieval():
    if request.method == 'GET':
        raw_html = f"Hi, User!<br>"
        # REMEMBER POST must be written post, with lower case letters
        raw_html += f'<form action="/get_entry/" method="post">'
        raw_html += '<label for="fname">entry id:</label><br>'
        raw_html += '<input type="text" id="entry_id" name="entry_id"><br>'
        raw_html += '<label for="fname">assembly id:</label><br>'
        raw_html += '<input type="text" id="assembly_id" name="assembly_id"><br>'
        raw_html += '<input type="submit" value="Submit">'
        raw_html += '</form>'
        return raw_html
    if request.method == 'POST':
        # Inside here I put my data and forward a request to the API
        entry_id = request.form.get('entry_id') # a multidict containing POST data
        assembly_id = request.form.get('assembly_id')
        result = rq.get(f'https://data.rcsb.org/rest/v1/core/assembly/{entry_id}/{assembly_id}').json()        
        return f"{result['pdbx_struct_assembly']}"
    if request.method == 'DELETE':
        return "This is dangerous"
    else:
        return "Error 405 Method Not Allowed"

In [ ]:
# Beware that this is really unstable, copy your app into a file and from terminal
# flask --app name_of_the_file run --host="0.0.0.0" [without the .py]
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple("0.0.0.0", 5000, app, use_debugger=True)


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.88.0.13:5000
Press CTRL+C to quit
192.168.127.1 - - [13/Oct/2024 18:22:46] "GET /get_entry/ HTTP/1.1" 200 -
192.168.127.1 - - [13/Oct/2024 18:22:46] "GET /favicon.ico HTTP/1.1" 404 -
192.168.127.1 - - [13/Oct/2024 18:22:46] "GET /get_entry/ HTTP/1.1" 200 -
192.168.127.1 - - [13/Oct/2024 18:22:50] "POST /get_entry/ HTTP/1.1" 200 -
